In [4]:
# load env ------------------------------------------------------------------------
import os
import utils

utils.load_env()
os.environ['LANGCHAIN_TRACING_V2'] = "false"


# debug ------------------------------------------------------------------
from langchain.globals import set_debug, set_verbose
set_verbose(True)
set_debug(False)

import functools
# for llm model
from langchain_openai import ChatOpenAI
# from langchain_community.chat_models import ChatOpenAI
from langchain_core.messages import (
    AIMessage, 
    HumanMessage,
    ToolMessage
)
from langgraph.graph import END, StateGraph, START
from tools import (
    find_place_from_text, 
    nearby_search, 
    nearby_dense_community, 
    search_population_community_household_expenditures_data,
    duckduckgo_search,
    get_tools_output,
    restaurant_sale_project
)
from agents import(
    create_agent,
    AgentState
)
from chat_history import save_chat_history, load_chat_history

## tools and LLM
# Bind the tools to the model
tools = [restaurant_sale_project, search_population_community_household_expenditures_data, find_place_from_text, nearby_search, nearby_dense_community, duckduckgo_search]  # Include both tools if needed

llm = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18", 
    temperature=0, 
    top_p=0.0, 
    )

# Helper function to create a node for a given agent
def agent_node(state, agent, name):
    result = agent.invoke(state)
    # We convert the agent output into a format that is suitable to append to the global state
    if isinstance(result, ToolMessage):
        pass
    else:
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
        # result = AIMessage(**result.dict(), name=name)
    return {
        "messages": [result],
        # Since we have a strict workflow, we can
        # track the sender so we know who to pass to next.
        "sender": name,
    }


## Define Agents Node ------------------------------------------------------------------------
# Research agent and node
from prompt import agent_meta
agent_name = [meta['name'] for meta in agent_meta]

# TODO: move agents to agents.py 
agents={}
agent_nodes={}

for meta in agent_meta:
    name = meta['name']
    prompt = meta['prompt']
    
    agents[name] = create_agent(
            llm,
            tools,
            system_message=prompt,
        )
    
    agent_nodes[name] = functools.partial(agent_node, agent=agents[name], name=name)


## Define Tool Node
from langgraph.prebuilt import ToolNode
from typing import Literal

tool_node = ToolNode(tools)

def router(state) -> Literal["call_tool", "__end__", "continue"]:
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if "continue" in last_message.content:
        return "continue"
    if last_message.tool_calls:
        # The previous agent is invoking a tool
        return "call_tool"
    if "%SIjfE923hf" in last_message.content:
        # Any agent decided the work is done
        return "__end__"
    else:
        return "continue"


## Workflow Graph ------------------------------------------------------------------------
workflow = StateGraph(AgentState)

# add agent nodes
for name, node in agent_nodes.items():
    workflow.add_node(name, node)
    
workflow.add_node("call_tool", tool_node)


workflow.add_conditional_edges(
    "analyst",
    router,
    {"continue": "data_collector", "call_tool": "call_tool", "__end__": END}
)

workflow.add_conditional_edges(
    "data_collector",
    router,
    {"call_tool": "call_tool", "continue": "reporter", "__end__": END}
)

workflow.add_conditional_edges(
    "reporter",
    router,
    {"continue": "data_collector", "call_tool": "call_tool", "__end__": END}
)

workflow.add_conditional_edges(
    "call_tool",
    # Each agent node updates the 'sender' field
    # the tool calling node does not, meaning
    # this edge will route back to the original agent
    # who invoked the tool
    lambda x: x["sender"],
    {name:name for name in agent_name},
)

workflow.add_edge(START, "analyst")
graph = workflow.compile()


def submitUserMessage(user_input: str, user_id:str="test", keep_chat_history=False, return_reference:bool=False, verbose=False) -> str:
    
    chat_history = load_chat_history(user_id=user_id) if keep_chat_history else []
    
    print(chat_history)
    
    graph = workflow.compile()

    events = graph.stream(
        {
            "messages": [
                HumanMessage(
                    user_input
                )
            ],
            "chat_history": chat_history
        },
        # Maximum number of steps to take in the graph
        {"recursion_limit": 20},
    )
    
    if not verbose:
        events = [e for e in events]
        response = list(events[-1].values())[0]
    else:
        for e in events:
            # print(e)
            a = list(e.items())[0]
            a[1]['messages'][0].pretty_print()
        
        response = a[1]
    
    
    
    save_chat_history(bot_response=response, user_id=user_id)
    
    response = response["messages"][0].content
    response = response.replace("%SIjfE923hf", "")
    
    if return_reference:
        return response, get_tools_output()
    else:
        return response

In [8]:
# from IPython.display import Image, display

# try:
#     display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
# except Exception:
#     # This requires some extra dependencies and is optional
#     pass

In [9]:
# import utils
submitUserMessage = utils.notify_process(submitUserMessage, sound_effect="purr")

result = submitUserMessage("วิเคราะห์ร้านแซนวิชแถวลุมพินี เซ็นเตอร์ ลาดพร้าว", keep_chat_history=True, return_reference=True, verbose=True)

================================== Ai Message ==================================
Name: analyst
Tool Calls:
  find_place_from_text (call_hfGgyjeM9HT50ouVopPEucY5)
 Call ID: call_hfGgyjeM9HT50ouVopPEucY5
  Args:
    location: ลุมพินี เซ็นเตอร์ ลาดพร้าว
  nearby_search (call_WN8hsbkuw9SKr8h8pKz3WZzQ)
 Call ID: call_WN8hsbkuw9SKr8h8pKz3WZzQ
  Args:
    keyword: sandwich shop
    location_name: ลุมพินี เซ็นเตอร์ ลาดพร้าว
================================= Tool Message =================================
Name: find_place_from_text


    address: 60 127 ซอย ลาดพร้าว 111 แขวงคลองจั่น เขตบางกะปิ กรุงเทพมหานคร 10240 ไทย

    location_name: ลุมพินี เซ็นเตอร์ ลาดพร้าว 111

    
================================== Ai Message ==================================
Name: analyst

I found the location details for "ลุมพินี เซ็นเตอร์ ลาดพร้าว." However, it seems there was an issue retrieving nearby sandwich shops. 

Here are the details for the location:
- **Address**: 60 127 ถนนลาดพร้าว 111, เขตลาดพร้าว, กรุงเ

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


================================= Tool Message =================================
Name: duckduckgo_search

Subway, 189 among Bangkok fast food: 103 reviews by visitors and 32 detailed photos. ... Bangkok / Subway, บางกะปิ 1 Soi Ban Lat Phrao; Subway. Add to wishlist. Add to compare. Share 189 of 9396 fast food in Bangkok ... There is a promotion during this time too Service: Take out Meal type: Breakfast Price per person: ฿100 ... MIAMI, Aug. 23, 2024 /PRNewswire/ -- Subway introduced a new deal as big as its iconic footlong sandwiches, perfect for anyone hungry for value, quality and variety. Starting August 26 through September 8, guests can order any footlong sub on Subway's menu for just $6.99*, whether it's one of the 22 chef-crafted subs in the Subway Series or a custom creation using Subway's signature pantry of ... Here are some more Subway codes that are active from time to time: BOGOD
================================== Ai Message ==================================
Name: data_c

In [2]:
submitUserMessage("report a sale projection of sandwish", keep_chat_history=True, return_reference=True, verbose=True)

================================== Ai Message ==================================
Name: analyst
Tool Calls:
  restaurant_sale_project (call_noXXzf94jjKTHXQ11zXKWPEK)
 Call ID: call_noXXzf94jjKTHXQ11zXKWPEK
  Args:
    input_dict: {'base_price': 300, 'category': 'Sandwich'}
================================= Tool Message =================================
Name: restaurant_sale_project

sale projection of Sandwich:
week	sale(forecast)
1	139,410
5	133,770
52	175,170

================================== Ai Message ==================================
Name: analyst

%SIjfE923hf

### Sales Projection Report for Sandwiches

Here is the forecasted sales projection for sandwiches based on a base price of 300 baht:

| Week | Sales (Forecast) |
|------|------------------|
| 1    | 139,410 baht     |
| 5    | 133,770 baht     |
| 52   | 175,170 baht     |

This projection indicates a strong potential for sales, especially as the year progresses. If you need further analysis or details, feel free to ask!

('\n\n### Sales Projection Report for Sandwiches\n\nHere is the forecasted sales projection for sandwiches based on a base price of 300 baht:\n\n| Week | Sales (Forecast) |\n|------|------------------|\n| 1    | 139,410 baht     |\n| 5    | 133,770 baht     |\n| 52   | 175,170 baht     |\n\nThis projection indicates a strong potential for sales, especially as the year progresses. If you need further analysis or details, feel free to ask!',
 'sale projection of Sandwich:\nweek\tsale(forecast)\n1\t139,410\n5\t133,770\n52\t175,170\n\n')